## Switch to CPU Instance (Advisable only for Non Colab-Pro instance)

1. Switch to CPU Instance for until Step 2 for non GPU dependent tasks
2. This increases your time available for the GPU dependent tasks on a Colab instance
2. Change Runtime type to CPU by Runtime(Top Left tab)->Change Runtime Type->None(Hardware Accelerator)
3.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Gaze Estimation using TAO GazeNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained model and train a GazeNet model on subset of MPIIFaceGaze dataset
* Run Inference on the trained model
* Export the retrained model to a .etlt file for deployment to DeepStream SDK

### Table of Contents

This notebook shows an example of gaze estimation using GazeNet in the Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables](#head-0)
1. [Prepare dataset and pre-trained model](#head-1) <br>
    1.1 [Verify downloaded dataset](#head-1-1) <br>
    1.2 [Convert datasets and labels to required format](#head-1-2) <br>
    1.3 [Verify dataset generation](#head-1-3) <br>
    1.4 [Download pre-trained model](#head-1-4) <br>
2. [Setup GPU environment](#head-2) <br>
    2.1 [Connect to GPU Instance](#head-2-1) <br>
    2.2 [Mounting Google drive](#head-2-2) <br>
    2.3 [Setup Python environment](#head-2-3) <br>
    2.4 [Reset env variables](#head-2-4) <br>
3. [Generate tfrecords from labels in json format](#head-3)
4. [Provide training specification](#head-4)
5. [Run TAO training](#head-5)
6. [Evaluate trained models](#head-6)
7. [Run inference for a set of images](#head-7)

## 0. Set up env variables <a class="anchor" id="head-0"></a>
When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env EXPERIMENT_DIR=/results/gazenet
%env DATA_DIR=/content/drive/MyDrive/emotionnet_data
%env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/tensorflow/gazenet/specs

# Showing list of specification files.
!ls -rlt $SPECS_DIR

## 1. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

This notebook uses a subset of MPIIFaceGaze dataset to illustrate the input data format for GazeNet and the procedures to use the generated data.

Please download the MPIIFaceGaze dataset from the following website:
https://www.mpi-inf.mpg.de/departments/computer-vision-and-machine-learning/research/gaze-based-human-computer-interaction/its-written-all-over-your-face-full-face-appearance-based-gaze-estimation

The labels for this subset based on required json format can be obtained from:
`$SAMPLES_DIR/gazenet/sample_labels`

In [ ]:
# check if the label file is presented
!if [ ! -f $DATA_DIR/data_factory.zip ]; then echo 'Label file not found, please check your sample path.'; else echo 'Found label file.';fi

After downloading the data, please unzip it to the `MPIIFaceGaze` folder and place the folder in `$DATA_DOWNLOAD_DIR`

After downloading the labels, please unzip it to the `data_factory` folder and place the folder in `MPIIFaceGaze`

You will then have the following path
* input data in `$LOCAL_DATA_DIR/MPIIFaceGaze`
* labels in `$LOCAL_DATA_DIR/MPIIFaceGaze/data_factory`

### A. Verify downloaded dataset <a class="anchor" id="head-1-1"></a>

In [ ]:
# Check the dataset is present
!if [ ! -d $DATA_DIR/MPIIFaceGaze ]; then echo 'Data folder not found, please download.'; else echo 'Found Data folder.';fi
!if [ ! -d $DATA_DIR/MPIIFaceGaze/data_factory ]; then echo 'Label folder not found, please download.'; else echo 'Found Labels folder.';fi

In [ ]:
# Sample json label.
!sed -n 1,201p $DATA_DIR/MPIIFaceGaze/data_factory/day03/p01/p01_day03.json

### B. Convert datasets and labels to required format <a class="anchor" id="head-1-2"></a>

A script is provided to convert the subset of `MPIIFaceGaze` dataset and downloaded labels to a required folder structure and dataset format.

In [ ]:
!python3.6 -m pip install opencv-python==4.2.0.32

In [ ]:
%cd /content/drive/MyDrive/ColabNotebooks/tensorflow/gazenet
!python3.6 mpiifacegaze_convert.py --data_path $DATA_DIR/MPIIFaceGaze \
                                 --json_label_root_path $DATA_DIR/MPIIFaceGaze

### C. Verify dataset generation <a class="anchor" id="head-1-3"></a>

A dataset folder with above-mentioned subset is created. All the required data to run GazeNet is saved under this folder.
* Generated data folder in `$LOCAL_DATA_DIR/MPIIFaceGaze/sample-dataset/p01-day03`
* Generated inference data folder in `$LOCAL_DATA_DIR/MPIIFaceGaze/sample-dataset/inference-set`

The converted dataset should have the following structure.

* `Config` folder in `$LOCAL_DATA_DIR/MPIIFaceGaze/sample-dataset/p01-day03/Config`
* `Data` folder in `$LOCAL_DATA_DIR/MPIIFaceGaze/sample-dataset/p01-day03/Data`
* `Labels` folder in `$LOCAL_DATA_DIR/MPIIFaceGaze/sample-dataset/p01-day03/json_datafactory_v2`

The inference dataset should have the following structure.

* `Config` folder in `$LOCAL_DATA_DIR/MPIIFaceGaze/sample-dataset/inference-set/Config`
* `Data` folder in `$LOCAL_DATA_DIR/MPIIFaceGaze/sample-dataset/inference-set/Data`
* `Labels` folder in `$LOCAL_DATA_DIR/MPIIFaceGaze/sample-dataset/inference-set/json_datafactory_v2`

In [ ]:
# Check the generated data is present
!if [ ! -d $DATA_DIR/MPIIFaceGaze/sample-dataset/p01-day03 ]; then echo 'Generated data folder not found, please regenerated.'; else echo 'Found generated data folder.';fi
!if [ ! -d $DATA_DIR/MPIIFaceGaze/sample-dataset/p01-day03/Config ]; then echo 'Config folder not found, please regenerated.'; else echo 'Found Config folder.';fi
!if [ ! -d $DATA_DIR/MPIIFaceGaze/sample-dataset/p01-day03/Data ]; then echo 'Data folder not found, please regenerated.'; else echo 'Found Data folder.';fi
!if [ ! -d $DATA_DIR/MPIIFaceGaze/sample-dataset/p01-day03/json_datafactory_v2 ]; then echo 'Labels folder not found, please regenerated.'; else echo 'Found Labels folder.';fi

# Check the inference data is present
!if [ ! -d $DATA_DIR/MPIIFaceGaze/sample-dataset/inference-set ]; then echo 'Inference data folder not found, please regenerated.'; else echo 'Found inference data folder.';fi
!if [ ! -d $DATA_DIR/MPIIFaceGaze/sample-dataset/inference-set/Config ]; then echo 'Config folder not found, please regenerated.'; else echo 'Found Config folder.';fi
!if [ ! -d $DATA_DIR/MPIIFaceGaze/sample-dataset/inference-set/Data ]; then echo 'Data folder not found, please regenerated.'; else echo 'Found Data folder.';fi
!if [ ! -d $DATA_DIR/MPIIFaceGaze/sample-dataset/inference-set/json_datafactory_v2 ]; then echo 'Labels folder not found, please regenerated.'; else echo 'Found Labels folder.';fi

### D. Download pre-trained model <a class="anchor" id="head-1-4"></a>

Please follow the instructions in the following to download and verify the pretrained model for gazenet.

For GazeNet pretrained model please download model: `nvidia/tao/gazenet:trainable_v1.0`.

After downloading the pre-trained model, please place the files in `$LOCAL_EXPERIMENT_DIR/pretrain_models`
You will then have the following path

* pretrained model in `$LOCAL_EXPERIMENT_DIR/pretrain_models/gazenet_vtrainable_v1.0/model.tlt`

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/content/
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 /content/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
# List models available in the model registry.
!ngc registry model list nvidia/tao/gazenet:*

In [ ]:
# Create the target destination to download the model.
!mkdir -p $EXPERIMENT_DIR/pretrain_models/

In [ ]:
# Download the pretrained model from NGC
!ngc registry model download-version nvidia/tao/gazenet:trainable_v1.0 \
    --dest $EXPERIMENT_DIR/pretrain_models/

In [ ]:
!ls -rlt $EXPERIMENT_DIR/pretrain_models/gazenet_vtrainable_v1.0

In [ ]:
# Check the dataset is present
!if [ ! -f $EXPERIMENT_DIR/pretrain_models/gazenet_vtrainable_v1.0/model.tlt ]; then echo 'Pretrain model file not found, please download.'; else echo 'Found Pretrain model file.';fi

## 2. Setup GPU environment <a class="anchor" id="head-2"></a>


### 2.1 Connect to GPU Instance <a class="anchor" id="head-2-1"></a>

1. Move any data saved to the Colab Instance storage to Google Drive  
2. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
3.   Then click on Connect (Top Right)



### 2.2 Mounting Google drive <a class="anchor" id="head-2-2"></a>
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 2.3 Setup Python environment <a class="anchor" id="head-2-3"></a>
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
!sh /content/drive/MyDrive/ColabNotebooks/tensorflow/setup_env.sh

### 2.4 Reset env variables <a class="anchor" id="head-2-4"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env KEY=nvidia_tlt
%env NUM_GPUS=1
%env EXPERIMENT_DIR=/results/classification
%env DATA_DIR=/content/drive/MyDrive/tf_data/classification_data/

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/classification

%env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/tensorflow/classification/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

## 3. Generate tfrecords from labels in json format <a class="anchor" id="head-3"></a>
* Create the tfrecords using the dataset_convert command 

In [ ]:
!python3.6 -m pip install uff

In [ ]:
!gazenet dataset_convert -folder-suffix pipeline \
                             -norm_folder_name Norm_Data \
                             -sets p01-day03 \
                             -data_root_path $DATA_DIR/MPIIFaceGaze/sample-dataset

In [ ]:
!ls -rl $DATA_DIR/MPIIFaceGaze/sample-dataset

In [ ]:
# check the tfrecords are presented
!if [ ! -d $DATA_DIR/MPIIFaceGaze/sample-dataset/p01-day03/Ground_Truth_DataFactory_pipeline ]; then echo 'Tfrecords folder not found, please generate.'; else echo 'Found Tfrecords folder.';fi

## 4. Provide training specification <a class="anchor" id="head-4"></a>
* Tfrecords for the train datasets
    * In order to use the newly generated tfrecords for training, update the 'ground_truth_folder_name' and 'tfrecords_directory_path' parameters of 'dataset_info' section in the spec file at `$SPECS_DIR/gazenet_tlt_pretrain.yaml`
* Pre-trained model path
    * Update "pretrained_model_path" in the spec file at `$SPECS_DIR/gazenet_tlt_pretrain.yaml`
    * If you want to training from random weights with your own data, you can enter "null" for "pretrained_model_path" section
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.

In [ ]:
!cat $LOCAL_SPECS_DIR/gazenet_tlt_pretrain.yaml

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the output directory location for models

*Note: The training may take hours to complete. Also, the remaining notebook, assumes that the training was done in single-GPU mode. 



In [87]:
!gazenet train -e $SPECS_DIR/gazenet_tlt_pretrain.yaml \
                   -r $USER_EXPERIMENT_DIR/experiment_result/exp1 \
                   -k $KEY

2022-07-08 21:55:56.617672: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.

2022-07-08 21:56:02,355 [WARNING] tensorflow: From /home/tao-dev/.cache/dazel/_dazel_tao-dev/d37c8d608f4a1d45f001faf4c28b14cc/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/driveix/build_wheel.runfiles/ai_infra/driveix/mitgazenet/dataloader/augmentation_helper.py:22: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


2022-07-08 21:56:02,355 [WARNING] tensorflow: From /home/tao-dev/.cache/dazel/_dazel_tao-dev/d37c8d608f4a1d45f001faf4c28b14cc/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/driveix/build_wheel.runfiles/ai_infra/driveix/mitgazenet/dataloader/augmentation_helper.py:22: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


2022-07-08 21:56:02,359 [WARNING] tensorflow: From /home/tao-dev/.cac

In [ ]:
!ls -lh $LOCAL_EXPERIMENT_DIR/experiment_result/exp1

## 6. Evaluate the trained model <a class="anchor" id="head-6"></a>


In [ ]:
!tao gazenet evaluate -type kpi_testing \
                      -m $USER_EXPERIMENT_DIR/experiment_result/exp1 \
                      -e $SPECS_DIR/gazenet_tlt_pretrain.yaml \
                      -k $KEY

In [ ]:
!ls -lh $LOCAL_EXPERIMENT_DIR/experiment_result/exp1/KPI_TMP

## 7. Visualize Inference <a class="anchor" id="head-7"></a>

In [ ]:
!tao gazenet inference -e $SPECS_DIR/gazenet_tlt_pretrain.yaml \
                       -i $DATA_DOWNLOAD_DIR/MPIIFaceGaze/sample-dataset/inference-set \
                       -m $USER_EXPERIMENT_DIR/experiment_result/exp1/model.tlt \
                       -o $USER_EXPERIMENT_DIR/experiment_result/exp1 \
                       -k $KEY

In [ ]:
!ls -lh $LOCAL_EXPERIMENT_DIR/experiment_result/exp1/result.txt

In [ ]:
import sys
import cv2
import numpy as np
import os
import json
import IPython.display
import PIL.Image
from utils_gazeviz import load_cam_intrinsics,\
        get_landmarks_dict, visualize_frame

# load data
data_root_path = os.path.join(os.environ['LOCAL_DATA_DIR'],
                              'MPIIFaceGaze/sample-dataset/inference-set')
print(data_root_path)
# load calibration
config_path = os.path.join(data_root_path, 'Config')
calib = {}
camera_mat, distortion_coeffs = load_cam_intrinsics(config_path)
distortion_coeffs = distortion_coeffs[0:5]
calib['cam'] = camera_mat
calib['dist'] = distortion_coeffs

# load json files
json_file_folder = os.path.join(data_root_path, 'json_datafactory_v2')
landmarks_dict = get_landmarks_dict(json_file_folder)
assert len(landmarks_dict.keys()) > 0

# visualize each frame in the result file
num_viz_frames = 5
result_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'],
                           "experiment_result/exp1/result.txt")

with open(result_path, 'r') as reader:
    lines = reader.readlines()

num_lines = len(lines)
num_viz_frames = min(num_viz_frames, num_lines)
for k in range(0, num_viz_frames):
    content = lines[k]
    line_info = content.split(' ')
    old_frame_path = line_info[0]
    sub_path = old_frame_path.split(os.environ['DATA_DOWNLOAD_DIR'])[-1]
    frame_path = os.environ['LOCAL_DATA_DIR'] + sub_path
    cam_coord = np.array(line_info[1:4], dtype=np.float32)
    frame_name = frame_path.split('/')[-1]
    landmarks_2D = landmarks_dict[frame_name]
    display_frame, le_px, le_por, re_px, re_por = visualize_frame(frame_path, landmarks_2D, cam_coord, calib)
    # Visualize selected landmarks
    cv2.arrowedLine(display_frame, tuple(le_px), tuple(le_por), (0, 255, 0), thickness=2, tipLength=0.05)
    cv2.arrowedLine(display_frame, tuple(re_px), tuple(re_por), (0, 255, 0), thickness=2, tipLength=0.05)
    IPython.display.display(PIL.Image.fromarray(display_frame))